In [2]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

In [3]:
resp = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
resp

<Response [200]>

In [4]:
soup = bs(resp.text, 'html.parser')

In [77]:
#soup.table.tbody.tr.
table = soup.table.tbody
this_row = table.tr
c1 = this_row.th
c2 = c1.next_sibling.next_sibling
c3 = c2.next_sibling.next_sibling

this_row = this_row.find_next_siblings("tr")
this_row[:3]

[<tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>]

In [59]:
rows = []
table = soup.table.tbody
respRows = soup.table.tbody.tr.find_next_siblings("tr")
c = 0
#ref=["Postal Code", "Borough", "Neighborhood"]

for row in respRows:
    newRow = {}
    cols = row.find_all("td")
    j = 0
    for col in cols:
        newRow[["Postal Code", "Borough", "Neighborhood"][j]] = col.text.replace('\n','')
        j+=1
    rows.append(newRow)
    
wikiData = pd.DataFrame(rows)
wikiData

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [60]:
wikiData_copy = wikiData

In [64]:
wikiData = wikiData[wikiData["Borough"]!="Not assigned"]
if len(wikiData[wikiData["Neighborhood"]=="Not assigned"]) == 0:
    print("No Missing Data! :D")
wikiData

No Missing Data! :D


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [70]:
geo1 = pd.read_csv("Geospatial_Coordinates.csv")
wikiData = pd.merge(left=wikiData, right=geo1, how='left', on="Postal Code")

In [71]:
wikiData

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [78]:
wikiData.to_csv("CanadaNeighborhoods.csv")